In [1]:
import mxnet as mx
mnist = mx.test_utils.get_mnist()

In [6]:
batch_size = 100
train_iter = mx.io.NDArrayIter(mnist['train_data'], mnist['train_label'], batch_size, shuffle=True)
val_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)

# Multilayer Perceptron model

In [4]:
data = mx.sym.var('data')
data = mx.sym.flatten(data=data)

fc1 = mx.sym.FullyConnected(data=data, num_hidden=128)
act1 = mx.sym.Activation(data=fc1, act_type="relu")

fc2 = mx.sym.FullyConnected(data=act1, num_hidden=64)
act2 = mx.sym.Activation(data=fc2, act_type="relu")

fc3 = mx.sym.FullyConnected(data=act2, num_hidden=10)
mlp = mx.sym.SoftmaxOutput(data=fc3, name='softmax')

In [7]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

mlp_model = mx.mod.Module(symbol=mlp, context=mx.cpu())

mlp_model.fit(train_iter,
              eval_data=val_iter,
              optimizer='sgd',
              optimizer_params={'learning_rate': 0.1},
              eval_metric='acc',
              batch_end_callback=mx.callback.Speedometer(batch_size, 100),
              num_epoch=10)

INFO:root:Epoch[0] Batch [0-100]	Speed: 95363.67 samples/sec	accuracy=0.112178
INFO:root:Epoch[0] Batch [100-200]	Speed: 88627.47 samples/sec	accuracy=0.112700
INFO:root:Epoch[0] Batch [200-300]	Speed: 101820.55 samples/sec	accuracy=0.113800
INFO:root:Epoch[0] Batch [300-400]	Speed: 103050.10 samples/sec	accuracy=0.109900
INFO:root:Epoch[0] Batch [400-500]	Speed: 103037.19 samples/sec	accuracy=0.126100
INFO:root:Epoch[0] Train-accuracy=0.134700
INFO:root:Epoch[0] Time cost=0.637
INFO:root:Epoch[0] Validation-accuracy=0.339300
INFO:root:Epoch[1] Batch [0-100]	Speed: 87347.67 samples/sec	accuracy=0.471584
INFO:root:Epoch[1] Batch [100-200]	Speed: 103002.52 samples/sec	accuracy=0.674100
INFO:root:Epoch[1] Batch [200-300]	Speed: 92681.36 samples/sec	accuracy=0.768500
INFO:root:Epoch[1] Batch [300-400]	Speed: 98520.05 samples/sec	accuracy=0.798900
INFO:root:Epoch[1] Batch [400-500]	Speed: 93285.53 samples/sec	accuracy=0.819900
INFO:root:Epoch[1] Train-accuracy=0.728700
INFO:root:Epoch[1] Ti

Generate predictions for entire test set:

In [11]:
test_iter = mx.io.NDArrayIter(mnist['test_data'], None, batch_size)
prob = mlp_model.predict(test_iter)
assert prob.shape == (10000, 10)

Calculate accuracy over test set:

In [12]:
test_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)
acc = mx.metric.Accuracy()
mlp_model.score(test_iter, acc)
print(acc)
assert acc.get()[1] > 0.96

EvalMetric: {'accuracy': 0.9714}


# Convolutional Model

In [13]:
data = mx.sym.var('data')

conv1 = mx.sym.Convolution(data=data, kernel=(5, 5), num_filter=20)
tanh1 = mx.sym.Activation(data=conv1, act_type='tanh')
pool1 = mx.sym.Pooling(data=tanh1, pool_type='max', kernel=(2, 2), stride=(2, 2))

conv2 = mx.sym.Convolution(data=pool1, kernel=(5, 5), num_filter=50)
tanh2 = mx.sym.Activation(data=conv2, act_type='tanh')
pool2 = mx.sym.Pooling(data=tanh2, pool_type='max', kernel=(2, 2), stride=(2, 2))

flatten = mx.sym.flatten(data=pool2)

fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=500)
tanh3 = mx.sym.Activation(data=fc1, act_type='tanh')

fc2 = mx.sym.FullyConnected(data=tanh3, num_hidden=10)

lenet = mx.sym.SoftmaxOutput(data=fc2, name='softmax')

In [16]:
lenet_model = mx.mod.Module(symbol=lenet, context=mx.cpu())

lenet_model.fit(train_iter,
                eval_data=val_iter,
                optimizer='sgd',
                optimizer_params={'learning_rate': 0.1},
                eval_metric='acc',
                batch_end_callback = mx.callback.Speedometer(batch_size, 100),
                num_epoch=10)

INFO:root:Epoch[0] Batch [0-100]	Speed: 2362.37 samples/sec	accuracy=0.120495
INFO:root:Epoch[0] Batch [100-200]	Speed: 2388.20 samples/sec	accuracy=0.108100
INFO:root:Epoch[0] Batch [200-300]	Speed: 2389.51 samples/sec	accuracy=0.111000
INFO:root:Epoch[0] Batch [300-400]	Speed: 2258.56 samples/sec	accuracy=0.107300
INFO:root:Epoch[0] Batch [400-500]	Speed: 2236.45 samples/sec	accuracy=0.113100
INFO:root:Epoch[0] Train-accuracy=0.112267
INFO:root:Epoch[0] Time cost=25.820
INFO:root:Epoch[0] Validation-accuracy=0.113500
INFO:root:Epoch[1] Batch [0-100]	Speed: 2356.28 samples/sec	accuracy=0.120495
INFO:root:Epoch[1] Batch [100-200]	Speed: 2296.11 samples/sec	accuracy=0.508500
INFO:root:Epoch[1] Batch [200-300]	Speed: 2270.93 samples/sec	accuracy=0.847100
INFO:root:Epoch[1] Batch [300-400]	Speed: 2293.47 samples/sec	accuracy=0.893000
INFO:root:Epoch[1] Batch [400-500]	Speed: 2306.17 samples/sec	accuracy=0.917800
INFO:root:Epoch[1] Train-accuracy=0.702500
INFO:root:Epoch[1] Time cost=26.07

In [17]:
test_iter = mx.io.NDArrayIter(mnist['test_data', None, batch_size])
prob = lenet_model.predict(test_iter)

test_iter = mx.io.NDArrayIter(mnist['test_data', mnist['test_label'], batch_size])
acc = mx.metric.Accuracy()
lenet_model.score(test_iter, acc)
print(acc)
assert acc.get()[1] > 0.98

KeyError: ('test_data', None, 100)